In [1]:
%matplotlib inline

In [2]:
import pandas as pd
import numpy as np
import os
import json
from pandas.io.json import json_normalize
import cPickle as pk
import matplotlib.pyplot as plt
import seaborn as sns

In [3]:
from IPython.core.display import HTML
css = open('style-table.css').read() + open('style-notebook.css').read()
HTML('<style>{}</style>'.format(css))

In [231]:
handle=open('../data/Kenya.pickle','r')
df2=pk.load(handle)
handle.close()
# comment line 2

In [232]:
df2.shape

(98099, 47)

In [316]:
def gender_extract(recip):
    lst=[]
    for i in recip:
        lst.append(i['gender'])
    return lst
        

In [364]:
proportion_fem=[]
borrower_team_size=[]

In [355]:
                                                   #This function is used in .apply() to fill two lists with
def fl_gender(recip,proportion_fem,borrower_team_size):
    sexes=gender_extract(recip)
    num_borrowers=len(sexes)
    #print 'sexes = ', sexes, 'num_borrowers = ',num_borrowers
    proportion_fem.append(float(sexes.count('F'))/num_borrowers)
    borrower_team_size.append(num_borrowers)

In [365]:
                                                     # Here, the two lists are filled from df2.borrowers
df2.borrowers.apply(fl_gender, args=(proportion_fem,borrower_team_size));

In [369]:
df2.loc[:,'prop_female']= pd.Series(proportion_fem,index=df2.index)      #add the lists as columns to the dataframe

In [376]:
df2.loc[:,'num_borrowers']=pd.Series(borrower_team_size,index=df2.index) #here too

In [380]:
df2.columns

Index([                     u'activity',                 u'basket_amount',
            u'bonus_credit_eligibility',                     u'borrowers',
                             u'country',                  u'country_code',
       u'currency_exchange_loss_amount',                    u'delinquent',
                         u'description',                 u'funded_amount',
                         u'funded_date',                     u'geo.level',
                           u'geo.pairs',                      u'geo.type',
                                  u'id',                         u'image',
                      u'journal_totals',                     u'languages',
                        u'lender_count',                   u'loan_amount',
                            u'location',                          u'name',
                                 u'obs',                   u'paid_amount',
                           u'paid_date',                    u'partner_id',
                         

In [433]:
df2.terms.iloc[0]

{u'disbursal_amount': 88000,
 u'disbursal_currency': u'KES',
 u'disbursal_date': u'2015-07-20T07:00:00Z',
 u'loan_amount': 875,
 u'local_payments': [{u'amount': 7334, u'due_date': u'2015-08-17T07:00:00Z'},
  {u'amount': 7334, u'due_date': u'2015-09-21T07:00:00Z'},
  {u'amount': 7334, u'due_date': u'2015-10-19T07:00:00Z'},
  {u'amount': 7334, u'due_date': u'2015-11-16T08:00:00Z'},
  {u'amount': 7334, u'due_date': u'2015-12-21T08:00:00Z'},
  {u'amount': 7334, u'due_date': u'2016-01-18T08:00:00Z'},
  {u'amount': 7334, u'due_date': u'2016-02-15T08:00:00Z'},
  {u'amount': 7334, u'due_date': u'2016-03-21T07:00:00Z'},
  {u'amount': 7334, u'due_date': u'2016-04-18T07:00:00Z'},
  {u'amount': 7334, u'due_date': u'2016-05-16T07:00:00Z'},
  {u'amount': 7334, u'due_date': u'2016-06-20T07:00:00Z'},
  {u'amount': 7326, u'due_date': u'2016-07-20T07:00:00Z'}],
 u'loss_liability': {u'currency_exchange': u'shared',
  u'currency_exchange_coverage_rate': 0.1,
  u'nonpayment': u'lender'},
 u'repayment_inter

In [432]:
n_terms= json_normalize(df2,'terms')

TypeError: string indices must be integers

In [400]:
df2['geo.pairs'].iloc[2]

u'1 38'

In [395]:
x

0    1 38
Name: geo.pairs, dtype: object